In [10]:
!pip install tsfresh

In [11]:
!pip install sktime

In [12]:
import sktime
import numpy as np
import pandas as pd

In [13]:
namesCol = ['subject', 'group', 'condition', 'trial', 'channel']

for q in range(256):
  strVar = 'v' + str(q)
  namesCol.append(strVar)

df = pd.read_csv('/content/drive/MyDrive/ML Final Project/eeg_test_uci.csv', header = 0, names = namesCol)
col_name = list(df.columns)
trans_df = pd.DataFrame(data = df, columns = col_name)

# Get Y-target -df
eeg_Y_test = trans_df['condition']

# Drop target variable and get X-feature- df
eeg_X_test = trans_df.drop(['condition'], axis = 1)

In [14]:
namesCol = ['subject', 'group', 'condition', 'trial', 'channel']

for q in range(256):
  strVar = 'v' + str(q)
  namesCol.append(strVar)

df = pd.read_csv('/content/drive/MyDrive/ML Final Project/eeg_train_uci.csv', header = 0, names = namesCol)
col_name = list(df.columns)
trans_df = pd.DataFrame(data = df, columns = col_name)

# Get Y-target -df
eeg_Y_train = trans_df['condition']

# Drop target variable and get X-feature- df
eeg_X_train = trans_df.drop(['condition'], axis = 1)

In [15]:
# Check data split

print(eeg_X_train.shape, eeg_Y_train.shape, eeg_X_test.shape, eeg_Y_test.shape)

(38400, 260) (38400,) (38400, 260) (38400,)


In [16]:
eeg_X_test.head()

,subject,group,trial,channel,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,...,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255
0,co2a0000364,a,30,FP1,-3.550,-5.015,-5.503,-3.550,-0.621,1.821,2.309,0.844,0.844,3.286,7.680,10.122,9.145,5.239,0.844,-0.621,2.309,6.215,7.192,4.262,-1.109,-5.503,-6.480,-4.038,-0.132,1.821,1.821,0.356,-0.132,1.821,4.262,6.704,7.192,5.727,4.262,3.774,...,8.169,8.657,8.169,7.680,7.680,7.192,6.704,6.215,5.239,4.262,3.774,4.751,6.704,8.169,8.657,6.704,2.309,-1.109,-3.062,-1.597,2.309,7.192,10.122,11.098,10.122,8.657,7.680,7.192,6.704,6.704,5.727,4.751,3.774,4.751,6.215,6.215,4.751,1.333,-1.109,0.356
1,co2a0000364,a,30,FP2,-4.476,-3.499,-1.058,1.383,3.337,2.848,2.360,2.848,4.801,7.731,8.708,7.731,3.825,-0.570,-3.011,-2.035,1.383,5.290,5.778,3.825,-0.570,-4.476,-5.452,-3.988,-2.035,-2.035,-3.499,-5.941,-6.429,-3.011,3.825,10.661,14.079,12.614,8.219,3.337,...,11.637,12.614,10.661,6.755,3.825,3.337,4.801,6.755,7.243,6.755,5.290,3.825,2.848,2.848,0.895,-1.546,-5.452,-7.406,-6.429,-2.035,5.290,11.637,14.079,12.126,7.243,1.872,0.407,2.360,6.755,9.684,8.219,2.848,-2.523,-4.476,-0.570,5.778,10.173,9.684,5.778,2.360
2,co2a0000364,a,30,F7,-1.017,-3.947,-5.900,-4.924,-2.970,-1.994,-1.994,-2.970,-1.017,4.354,9.725,13.631,12.655,6.795,1.424,-1.994,0.448,5.819,9.237,7.772,1.912,-5.412,-7.853,-4.435,3.377,9.237,9.237,5.330,-0.041,-1.994,-0.529,1.912,2.889,2.401,2.889,4.354,...,3.377,5.819,10.701,14.119,12.655,8.260,2.889,-1.017,-1.506,-1.506,-1.994,-2.482,-1.017,3.866,9.725,12.166,8.260,0.448,-6.388,-6.877,-1.017,5.819,7.284,2.889,-3.947,-7.853,-4.435,4.354,12.655,15.096,11.678,4.842,-0.529,-1.994,-0.529,1.424,0.936,-2.970,-5.900,-6.388
3,co2a0000364,a,30,F8,-0.661,5.198,10.569,13.987,13.499,9.593,4.222,1.292,1.780,5.198,7.640,7.151,4.222,1.292,-0.173,-0.661,-1.638,-2.614,-2.126,1.292,6.175,9.104,8.616,1.780,-7.985,-17.751,-22.634,-20.192,-8.962,6.175,18.870,23.265,16.917,4.710,-4.079,-4.079,...,4.710,9.593,11.058,7.151,0.804,-1.149,1.292,5.686,6.663,2.757,-3.103,-5.056,-2.126,2.757,3.733,-1.149,-10.915,-17.263,-15.310,-5.544,7.151,14.964,13.987,6.663,-2.126,-7.497,-7.009,-1.638,4.710,9.104,9.104,4.222,-2.614,-7.985,-7.985,-3.591,1.780,3.733,2.268,0.315
4,co2a0000364,a,30,AF1,-6.083,-4.618,-2.177,1.241,2.706,2.706,1.729,1.241,2.218,4.659,6.612,7.100,5.636,3.682,2.706,4.171,6.612,8.565,8.077,5.147,0.753,-2.665,-2.665,0.264,2.706,3.682,3.194,1.241,0.753,2.706,6.612,9.542,11.007,10.030,8.077,6.612,...,5.636,7.589,9.542,10.030,9.542,9.054,7.589,6.612,5.147,4.171,3.682,4.171,5.147,7.100,8.077,7.589,4.659,2.218,-0.224,-0.224,1.241,3.194,3.682,3.682,3.682,3.682,5.147,6.124,6.612,5.636,3.194,0.753,-0.712,0.264,1.241,1.729,0.264,-1.689,-2.177,-0.224


In [17]:
eeg_X_train.head()

,subject,group,trial,channel,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v25,v26,v27,v28,v29,v30,v31,v32,v33,v34,v35,...,v216,v217,v218,v219,v220,v221,v222,v223,v224,v225,v226,v227,v228,v229,v230,v231,v232,v233,v234,v235,v236,v237,v238,v239,v240,v241,v242,v243,v244,v245,v246,v247,v248,v249,v250,v251,v252,v253,v254,v255
0,co2a0000364,a,0,FP1,-8.921,-8.433,-2.574,5.239,11.587,14.028,11.587,6.704,1.821,-1.109,-2.085,-1.597,0.356,2.309,2.797,2.309,0.356,-0.132,-0.132,0.356,0.844,1.333,2.797,6.215,9.633,10.610,6.704,-0.621,-5.992,-6.480,-1.597,4.262,7.680,5.727,1.821,0.356,...,14.516,14.028,9.633,5.727,4.262,5.727,7.192,7.192,5.727,4.751,6.704,10.610,13.051,13.051,9.633,6.704,6.704,8.657,10.610,9.145,4.751,-0.132,-2.574,-1.109,2.309,6.215,8.657,10.610,12.075,12.563,10.610,8.169,4.751,2.797,2.797,3.286,3.774,4.262,5.727,8.169
1,co2a0000364,a,0,FP2,0.834,3.276,5.717,7.670,9.623,9.623,8.647,5.229,1.322,-2.096,-4.049,-5.025,-5.025,-5.025,-4.537,-4.049,-1.607,2.299,6.205,8.647,9.135,7.670,6.205,5.717,6.205,4.252,-0.631,-7.955,-11.861,-10.396,-2.584,6.205,10.111,6.694,0.346,-2.096,...,12.553,9.135,3.764,-0.142,-0.631,2.787,6.694,8.647,8.158,6.694,6.205,6.694,6.694,5.229,2.787,3.276,6.694,10.600,11.576,7.182,-0.142,-5.514,-5.025,1.322,8.158,11.088,9.623,5.717,3.764,7.182,12.065,15.483,12.553,5.229,-2.584,-6.490,-4.049,2.787,9.623,13.041
2,co2a0000364,a,0,F7,-19.847,-12.522,1.149,14.821,20.681,17.263,8.962,0.173,-3.733,-2.268,0.661,2.614,3.103,2.614,3.103,4.567,7.009,7.985,7.009,4.079,0.173,-2.268,-0.804,3.591,8.962,11.403,9.939,4.567,-1.780,-5.198,-3.245,2.614,8.962,11.403,8.962,2.614,...,18.728,21.169,17.263,10.915,7.009,8.962,13.357,15.798,12.380,5.544,0.173,1.149,7.009,13.357,16.286,15.310,13.845,15.310,18.239,18.728,14.821,7.497,2.126,2.126,6.521,11.403,13.357,13.357,14.333,16.774,19.216,17.751,10.915,2.126,-4.222,-4.710,0.661,7.009,11.403,12.868
3,co2a0000364,a,0,F8,8.148,1.801,-2.594,-4.547,-5.035,-5.524,-4.547,-0.641,5.707,10.590,9.125,1.801,-6.500,-9.918,-6.500,-0.641,4.242,4.730,1.312,-2.594,-6.500,-7.477,-4.059,4.242,14.008,19.379,15.961,3.754,-9.918,-16.266,-12.360,-3.082,2.777,-0.641,-8.942,-11.871,...,4.730,9.613,11.078,9.125,5.707,2.777,0.336,-3.082,-6.012,-6.500,-2.106,5.219,10.101,7.660,-1.129,-9.430,-10.406,-2.106,10.101,17.914,15.961,7.172,-2.106,-6.500,-5.524,-4.059,-3.571,-3.082,0.336,8.148,16.449,20.355,16.937,7.660,-2.594,-9.430,-10.406,-7.477,-1.617,4.242
4,co2a0000364,a,0,AF1,-2.146,-2.146,-1.658,-0.682,2.248,5.178,6.643,5.178,1.760,-1.658,-3.611,-1.658,1.272,3.225,2.736,0.295,-1.658,-0.682,1.760,3.713,3.225,2.248,2.736,4.690,6.643,6.643,3.225,-1.658,-4.100,-3.123,0.295,3.713,4.690,2.736,0.783,1.272,...,10.549,10.549,7.131,2.736,0.295,0.295,2.248,5.666,7.131,7.131,5.178,3.225,2.248,3.225,4.201,5.666,6.154,4.201,2.248,0.295,-0.682,0.295,1.760,3.225,3.713,4.201,3.225,2.248,0.295,-1.170,-1.658,0.295,2.736,4.201,3.713,1.272,-0.682,-0.193,3.225,7.619


In [31]:
# encoding labels in X datasets
from sklearn import preprocessing

cat_cols = ['subject', 'group', 'trial']
# cat_cols = ['group', 'trial']
enc = preprocessing.LabelEncoder()

for col in cat_cols:
    eeg_X_train[col] = eeg_X_train[col].astype('str')
    eeg_X_test[col] = eeg_X_test[col].astype('str')
    eeg_X_train[col] = enc.fit_transform(eeg_X_train[col])
    eeg_X_test[col] = enc.transform(eeg_X_test[col]) # this throws a labeling error, I assume because of the subject IDs

ValueError: ignored

In [32]:
#isolate the time series
eeg_X_train_timedata = eeg_X_train[eeg_X_train.columns[4:260]]
eeg_X_test_timedata = eeg_X_test[eeg_X_test.columns[4:260]]



eeg_X_train_timedata['combine'] = eeg_X_train_timedata.values.tolist()
eeg_X_test_timedata['combine'] = eeg_X_test_timedata.values.tolist()
eeg_X_train_timedata = eeg_X_train_timedata['combine']
eeg_X_test_timedata = eeg_X_test_timedata['combine']

#convert to dataframe
eeg_X_train_timedata = eeg_X_train_timedata.to_frame()
eeg_X_test_timedata = eeg_X_test_timedata.to_frame()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
eeg_ts_train = pd.Series(eeg_X_train_timedata['combine'].values, index=eeg_X_train_timedata.index)
eeg_X_ts_train = eeg_ts_train.to_frame()

eeg_ts_test = pd.Series(eeg_X_test_timedata['combine'].values, index=eeg_X_test_timedata.index)
eeg_X_ts_test = eeg_ts_test.to_frame()

In [34]:
for row_num in range(0,eeg_X_ts_train.shape[0]):
  eeg_train_series = pd.Series(eeg_X_ts_train.iat[row_num,0])
  eeg_X_ts_train.iat[row_num,0] = eeg_train_series

for row_num in range(0,eeg_X_ts_test.shape[0]):
  eeg_test_series = pd.Series(eeg_X_ts_test.iat[row_num,0])
  eeg_X_ts_test.iat[row_num,0] = eeg_test_series

Verifying the format of the data

In [35]:
eeg_X_ts_test.head()

,0
0,0 -3.550 1 -5.015 2 -5.503 3 -...
1,0 -4.476 1 -3.499 2 -1.058 3 ...
2,0 -1.017 1 -3.947 2 -5.900 3 -...
3,0 -0.661 1 5.198 2 10.569 3 ...
4,0 -6.083 1 -4.618 2 -2.177 3 ...


In [36]:
eeg_X_ts_test.head()

,0
0,0 -3.550 1 -5.015 2 -5.503 3 -...
1,0 -4.476 1 -3.499 2 -1.058 3 ...
2,0 -1.017 1 -3.947 2 -5.900 3 -...
3,0 -0.661 1 5.198 2 10.569 3 ...
4,0 -6.083 1 -4.618 2 -2.177 3 ...


Proximity Forest

In [ ]:
from sktime.classification.distance_based import ProximityForest

prox_forest = ProximityForest(n_estimators=5)
prox_forest.fit(eeg_X_ts_train, eeg_Y_train)
prox_forest.score(eeg_X_ts_test, eeg_Y_test)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')